In [116]:
from pyspark.sql import SparkSession
from operator import add
from IPython.core.display import display, HTML
from pprint import pprint
import pyspark
from pyspark.sql.functions import udf, count, col 
from pyspark.sql.types import StringType
from pyspark.sql.functions import to_timestamp, dayofyear
import pyspark.sql.functions as f
from pyspark.sql import Window

In [117]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("hadoop_punnam_taskB")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",8)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [118]:
# Load the csv from HDFS
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/parking-citations.csv')\
    .cache()

In [119]:
# Verify that the data has loaded has correctly
data_frame.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|    99999|    99999|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|         

In [120]:
# Showing the schema of the dataframe
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [121]:
# Count the no. of rows
data_frame.count()

9257460

In [122]:
# No. of Partitions in RDD
data_frame.rdd.getNumPartitions()

10

In [123]:
# Dropping the columns
new_df = data_frame.drop('VIN','Latitude','Longitude')

In [124]:
# Printing the new schema
new_df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)



In [125]:
# Casting the string type fine values to float
temp_df = new_df.withColumn('NewFine',new_df['Fine amount'].cast("float"))

In [126]:
# Checking the new type
temp_df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- NewFine: float (nullable = true)



In [127]:
# Taking the top 10 tickets by their fine values
temp_df.orderBy('NewFine', ascending = False).take(10)

[Row(Ticket number='1109907271', Issue Date='2015-03-09T00:00:00', Issue time='610', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201510', Make='FORD', Body Style='PA', Color='RE', Location='1201 E 35TH ST', Route='13E3', Agency='1', Violation code='558', Violation Description='8755**', Fine amount='505', NewFine=505.0),
 Row(Ticket number='1109908925', Issue Date='2015-03-06T00:00:00', Issue time='2200', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201510', Make='FORD', Body Style='PA', Color='RE', Location='1203 E 35TH ST', Route='13E3', Agency='1', Violation code='558', Violation Description='8755**', Fine amount='505', NewFine=505.0),
 Row(Ticket number='1109909205', Issue Date='2014-12-24T00:00:00', Issue time='110', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201410', Make='HOND', Body Style=None, Color='BK', Location='6500 S AVALON BLVD', Route='13E3', Agency='1', Violation code='558', Violation

In [128]:
# Grouping tickets whose fine value is the maximum
temp_df.where(f.col('NewFine') == temp_df.orderBy('NewFine', ascending = False).take(1)[0]['NewFine']).count()

6

In [129]:
# Grouping the make of the cars and counting each group
cnts = temp_df.groupBy("Make").agg(count("*").alias("cnt"))
# Cumulating the count of each group
maxs = cnts.groupBy("Make").agg(f.max("cnt").alias("mx"))

In [130]:
# Displaying the top 20 make of the cars
maxs.orderBy('mx', ascending = False).take(20)

[Row(Make='TOYT', mx=1531949),
 Row(Make='HOND', mx=1043276),
 Row(Make='FORD', mx=807498),
 Row(Make='NISS', mx=662097),
 Row(Make='CHEV', mx=631413),
 Row(Make='BMW', mx=422916),
 Row(Make='MERZ', mx=376830),
 Row(Make='VOLK', mx=316002),
 Row(Make='HYUN', mx=285286),
 Row(Make='DODG', mx=271590),
 Row(Make='LEXS', mx=263269),
 Row(Make='KIA', mx=217795),
 Row(Make='JEEP', mx=214965),
 Row(Make='AUDI', mx=179718),
 Row(Make='MAZD', mx=169811),
 Row(Make='OTHR', mx=154376),
 Row(Make='GMC', mx=132788),
 Row(Make='INFI', mx=120340),
 Row(Make='CHRY', mx=120317),
 Row(Make='ACUR', mx=111265)]

In [131]:
COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

In [132]:
# Replacing the color shortcuts to the full name
color_df = temp_df.na.replace(COLORS, 1, 'Color')

/home/ubuntu/.local/lib/python3.6/site-packages/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [133]:
# Displaying the results
color_df.show(5)

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+------------------+-----+------+--------------+---------------------+-----------+-------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|          Location|Route|Agency|Violation code|Violation Description|Fine amount|NewFine|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+------------------+-----+------+--------------+---------------------+-----------+-------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA| Gray|   13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|   50.0|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|White|     525 S MAIN ST| 1C51|     

In [134]:
# Seeing whether tickets whose color isn't a key in our dictionary changes or not
# See 2nd Ticket
color_df.orderBy('NewFine', ascending = False).take(10)

[Row(Ticket number='1109907271', Issue Date='2015-03-09T00:00:00', Issue time='610', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201510', Make='FORD', Body Style='PA', Color='Red', Location='1201 E 35TH ST', Route='13E3', Agency='1', Violation code='558', Violation Description='8755**', Fine amount='505', NewFine=505.0),
 Row(Ticket number='1109908925', Issue Date='2015-03-06T00:00:00', Issue time='2200', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201510', Make='FORD', Body Style='PA', Color='Red', Location='1203 E 35TH ST', Route='13E3', Agency='1', Violation code='558', Violation Description='8755**', Fine amount='505', NewFine=505.0),
 Row(Ticket number='1109909205', Issue Date='2014-12-24T00:00:00', Issue time='110', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201410', Make='HOND', Body Style=None, Color='Black', Location='6500 S AVALON BLVD', Route='13E3', Agency='1', Violation code='558', Viol

In [135]:
# Tickets whose make is the maximum make i.e TOYT
new_color_df = color_df.where(f.col('Make') == maxs.orderBy('mx', ascending = False).take(1)[0]['Make'])

In [136]:
# Grouping the tickets by their color and counting
cnts_color = new_color_df.groupBy("Color").agg(count("*").alias("cnt"))
# Cumulating the count of each color 
maxs_color = cnts_color.groupBy("Color").agg(f.max("cnt").alias("mx"))

In [137]:
# Top color in the TOYT make group
maxs_color.orderBy('mx', ascending = False).take(10)

[Row(Color='Gray', mx=346822),
 Row(Color='White', mx=304620),
 Row(Color='Black', mx=252199),
 Row(Color='Silver', mx=248685),
 Row(Color='Blue', mx=128051),
 Row(Color='Red', mx=84175),
 Row(Color='Green', mx=57627),
 Row(Color='Gold', mx=30154),
 Row(Color='Maroon', mx=19882),
 Row(Color='Tan', mx=17006)]